In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(9,4.5))
rc('font',size=12)
rc('text',usetex=False)

#from qubicpack import qubicpack as qp
from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import satorchipy as stpy
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
import pickle
from importlib import reload

In [ ]:
day = '2019-11-15'
keyword = '**'
data_dir = '/qubic/Data/Calib-TD/'+day+'/'
#dirs = np.sort(glob.glob(data_dir+'*test_sw*'))
dirs = np.sort(glob.glob(data_dir+keyword))
print (dirs)

dirs = [dirs[4], dirs[6]]
print('')
print(dirs)
names = ['PT On', 'PT Off']

In [ ]:
TESNum = 96

if TESNum <=128:
    asic = 1
else:
    asic = 2

tt = []
dd = []
rmsk = []
for thedir in dirs:
    a = qubicfp()
    a.Rfeedback = 100e3 ### Apparent bug in QubicStudio... the value is wrong...
    a.read_qubicstudio_dataset(thedir)
    rmsk.append(a.RawMask(asic=asic))
    data=a.azel_etc(TES=None)
    t0=data['t_data {}'.format(asic)][0]
    tt.append(data['t_data {}'.format(asic)]-t0)
    dd_ADU = data['data {}'.format(asic)]
    dd.append(a.ADU2I(dd_ADU))


In [ ]:
subplot(2,1,1)
for i in range(len(dirs)):
    plot(tt[i], dd[i][TESNum-1,:], label = dirs[i])
title('All data')

lims = [[0,500],[0,50]]
subplot(2,1,2)
for i in range(len(dirs)):
    ok = (tt[i] > lims[i][0]) & (tt[i] < lims[i][1])
    tt[i] = tt[i][ok]
    dd[i] = dd[i][:,ok]
    plot(tt[i], dd[i][TESNum-1,:], label = dirs[i])
title('After Cut')

In [ ]:
rc('figure',figsize=(15,8))
TESNum = 95

xmin = 0.01
xmax = 100.
ymin = 1e-17
ymax = 1e-12

voltage = 1.4e-6

for i in range(len(dirs)):
    ############ Power spectrum
    spectrum_f, freq_f = ft.power_spectrum(tt[i], dd[i][TESNum-1,:]*1e-6, rebin=True)
    plot(freq_f, np.sqrt(f.gaussian_filter1d(spectrum_f,1)) * voltage,label=names[i])
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Amplitude Spectral Density $[W/\sqrt{Hz}]$')
xlim(xmin, xmax)
ylim(ymin, ymax)
title('TES#={}'.format(TESNum))
legend()
grid()

#plot([19,19],[ymin,ymax],'--')

In [ ]:
### Loop on TES
TESNum = 34
doplot = True
regions = [[0.1, 1.], [1., 10.], [10., 50.]]
meanps = np.zeros((len(dirs), len(regions), 256))

for i in range(len(dirs)):
    a = qubicfp()
    a.Rfeedback = 100e3 ### Apparent bug in QubicStudio... the value is wrong...
    a.read_qubicstudio_dataset(thedir)
    data=a.azel_etc(TES=None)
    for asic in [1,2]:
        print(i,asic)
        t0=data['t_data {}'.format(asic)][0]
        tt.append(data['t_data {}'.format(asic)]-t0)
        dd_ADU = data['data {}'.format(asic)]
        dd.append(a.ADU2I(dd_ADU))
        for k in range(128):
            spectrum_f, freq_f = ft.power_spectrum(tt[i], dd[i][k,:]*1e-6 * voltage, rebin=True)
            for r in range(len(regions)):
                ok = (freq_f >= regions[r][0]) & (freq_f < regions[r][1])
                mm, ss = ft.meancut(spectrum_f[ok],3)
                meanps[i,r,128*(asic-1)+k] = np.sqrt(mm)
            if doplot:
                if ((128*(asic-1)+k)+1)==TESNum:
                    p=plot(freq_f, np.sqrt(f.gaussian_filter1d(spectrum_f,1)),label=names[i])
                    yscale('log')
                    xscale('log')
                    xlabel('Frequency [Hz]')
                    ylabel('Amplitude Spectral Density $[W/\sqrt{Hz}]$')
                    xlim(xmin, xmax)
                    ylim(ymin, ymax)
                    title('TES#={}'.format(TESNum))
                    print(meanps[i,:,128*(asic-1)+k])
                    for r in range(len(regions)):
                        plot(regions[r], [meanps[i,r,128*(asic-1)+k], meanps[i,r,128*(asic-1)+k]])
                    


legend()
grid()
            
            
            

In [ ]:
idir = 0
for idir in range(2):
    for i in range(len(regions)):
        subplot(2,3,i+1+3*idir)
        thevalues = np.log10(meanps[idir, i, :])
        bad = thevalues < -18
        thevalues[bad] = np.nan
        img = ft.image_asics(all1=thevalues)
        imshow(img,vmin=-15.5,vmax=-14)
        colorbar()
        title('[{0:4.2f} , {1:4.2f}] Hz \n '.format(regions[i][0], regions[i][1])+names[idir])
tight_layout()

In [ ]:
for idir in range(2):
    for i in range(len(regions)):
        subplot(2,3,i+1+3*idir)
        thevalues = np.log10(meanps[idir, i, :])
        bad = thevalues < -18
        thevalues[bad] = np.nan
        mm, ss = ft.meancut(10**thevalues[np.isfinite(thevalues)], 3)
        print(mm, np.sum(np.isfinite(thevalues)))
        nep = mm
        hist(thevalues, range=[-17, -13], bins=21, label='<NEP> = {0:5.2g}'.format(nep))
        #hist(thevalues, range=[-17, -13], bins=21, label=ft.statstr(thevalues[np.isfinite(thevalues)], cut=3))
        title('[{0:4.2f} , {1:4.2f}] Hz \n '.format(regions[i][0], regions[i][1])+names[idir])
        xlabel('Log10(NEP [$W/\sqrt{Hz})$])')
        legend()
tight_layout()

In [ ]:
for idir in range(2):
    for i in range(len(regions)):
        subplot(2,3,i+1+3*idir)
        thevalues = meanps[idir, i, :]*1e17
        bad = np.log10(thevalues/1e17) < -18
        thevalues[bad] = np.nan
        mm, ss = ft.meancut(thevalues[np.isfinite(thevalues)], 3)
        print(mm/1e17, np.sum(np.isfinite(thevalues)))
        nep = mm/1e17
        hist(thevalues, range=[1,1000], bins=21, label='<NEP> = {0:5.2g}'.format(nep))
        title('[{0:4.2f} , {1:4.2f}] Hz \n '.format(regions[i][0], regions[i][1])+names[idir])
        xlabel('$10^{17}$ NEP [$W/\sqrt{Hz})$]')
        legend()
tight_layout()

In [ ]:
for idir in range(2):
    for i in range(len(regions)):
        subplot(2,3,i+1+3*idir)
        thevalues = meanps[idir, i, :]*1e17
        bad = np.log10(thevalues/1e17) < -18
        thevalues[bad] = np.nan
        mm, ss = ft.meancut(thevalues[np.isfinite(thevalues)], 3)
        print(mm/1e17, np.sum(np.isfinite(thevalues)))
        nep = mm/1e17
        hist(thevalues, range=[1,200], bins=21, label='<NEP> = {0:5.2g}'.format(nep))
        title('[{0:4.2f} , {1:4.2f}] Hz \n '.format(regions[i][0], regions[i][1])+names[idir])
        xlabel('$10^{17}$ NEP [$W/\sqrt{Hz})$]')
        legend()
tight_layout()

In [ ]:
for i in range(len(regions)):
    subplot(2,3,i+1+3*idir)
    thevalues = np.log10(meanps[0, i, :] / meanps[1, i, :])
    thevalues[bad] = np.nan
    img = ft.image_asics(all1=thevalues)
    imshow(img,vmin=-1, vmax=2)
    colorbar()
    title('[{0:4.2f} , {1:4.2f}] Hz \n '.format(regions[i][0], regions[i][1])+names[0]+'/'+names[1])

In [ ]:
for i in range(len(regions)):
    subplot(2,3,i+1+3*idir)
    thevalues = (meanps[0, i, :] / meanps[1, i, :])
    thevalues[bad] = np.nan
    hist(thevalues, range=[0,10],bins=31, label=ft.statstr(thevalues[np.isfinite(thevalues)], cut=3))
    title('[{0:4.2f} , {1:4.2f}] Hz \n Ratio '.format(regions[i][0], regions[i][1])+names[0]+'/'+names[1])
    legend()